In [1]:
import mmh3

In [2]:
import functools

In [3]:
import iteration_utilities

In [4]:
import random

In [5]:
import numpy as np

In [6]:
import math

In [7]:
import tensorflow as tf

In [8]:
from tensorflow.python import debug as tf_debug

In [9]:
from enum import IntEnum

In [10]:
print("TensorFlow version: {}".format(tf.VERSION))

TensorFlow version: 1.8.0


In [11]:
# (x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
training_data, testing_data = tf.keras.datasets.fashion_mnist.load_data()

In [12]:
number_of_classes = int(np.max(testing_data[1])) + 1
number_of_input_channels = 1

In [13]:
number_of_experts = 1

In [14]:
number_of_layers_per_block = 16

In [15]:
number_of_features_per_layer = 8

In [16]:
number_of_blocks = 3

In [17]:
number_of_features_of_bottleneck = 32

In [18]:
paddings = (0, 0), (2, 2), (2, 2)

In [19]:
def offset_initializer():
    return tf.random_normal_initializer(stddev=3.0)
#     return tf.random_uniform_initializer(minval=-3.0, maxval=3.0)

In [20]:
def grid_sample(images, offset_x, offset_y):
    tf.assert_rank(images, 4)
    tf.assert_rank(offset_x, 4)
    tf.assert_rank(offset_y, 4)
    image_shape = tf.shape(images)
    current_batch_size = image_shape[0]
    width = image_shape[1]
    height = image_shape[2]
    number_of_features = image_shape[3]
    
    def check_width(tensor):
        if (tf.test.is_gpu_available()):
            return tensor
        else:
            return tf.maximum(tf.minimum(tensor, width - 1), 0)
    
    def check_height(tensor):
        if (tf.test.is_gpu_available()):
            return tensor
        else:
            return tf.maximum(tf.minimum(tensor, height - 1), 0)
    
    offset_left = tf.floor(offset_x)
    offset_top = tf.floor(offset_y)
    tf.assert_equal(tf.shape(offset_left), image_shape)
    tf.assert_equal(tf.shape(offset_top), image_shape)

    
    eps = 1e-7

    factor_right = offset_x - offset_left + eps
    factor_left = 1.0 + 2.0 * eps - factor_right
    factor_bottom = offset_y - offset_top + eps
    factor_top = 1.0 + 2.0 * eps - factor_bottom
    
    image_index, x_index, y_index, feature_index = tf.meshgrid(
        tf.range(current_batch_size, dtype=tf.int32),
        tf.range(width, dtype=tf.int32),
        tf.range(height, dtype=tf.int32),
        tf.range(number_of_features, dtype=tf.int32),
        indexing='ij',
    )

    left_index = check_width(x_index + tf.cast(offset_left, tf.int32))
    top_index = check_height(y_index + tf.cast(offset_top, tf.int32))

    right_index = left_index + 1
    bottom_index = top_index + 1
    
    tf.assert_rank(left_index, 4)
    tf.assert_rank(right_index, 4)
    tf.assert_rank(bottom_index, 4)
    tf.assert_rank(top_index, 4)
    
    images_top_left = tf.gather_nd(images, tf.stack((image_index, left_index, top_index, feature_index), axis=4))
    images_top_right = tf.gather_nd(images, tf.stack((image_index, right_index, top_index, feature_index), axis=4))
    images_bottom_left = tf.gather_nd(images, tf.stack((image_index, left_index, bottom_index, feature_index), axis=4))
    images_bottom_right = tf.gather_nd(images, tf.stack((image_index, right_index, bottom_index, feature_index), axis=4))

    tf.assert_rank(images_top_left, 4)
    tf.assert_rank(images_top_right, 4)
    tf.assert_rank(images_bottom_left, 4)
    tf.assert_rank(images_bottom_right, 4)

    lerp_top = factor_left * images_top_left + factor_right * images_top_right
    lerp_bottom = factor_left * images_bottom_left + factor_right * images_bottom_right
    output = factor_top * lerp_top + factor_bottom * lerp_bottom
    tf.assert_rank(output, 4)
    return output

In [21]:
def offnet(images, number_of_features, number_of_experts=None, name=None):
    with tf.variable_scope(name, default_name="offnet"):
        input_shape = images.shape
        if number_of_experts is None:
            number_of_experts = int(input_shape[0])
        number_of_input_features = int(input_shape[4])
        score_weight = tf.get_variable(
            name="score_weight",
            initializer=tf.random_normal_initializer(stddev=tf.sqrt(1.0 / number_of_input_features)),
            dtype=tf.float32,
            shape=(number_of_experts, number_of_input_features, number_of_features)
        )
        offset_x_weight = tf.get_variable(
            name="offset_x_weight",
            initializer=tf.random_normal_initializer(stddev=tf.sqrt(1.0 / number_of_input_features)),
            dtype=tf.float32,
            shape=(number_of_experts, number_of_input_features, number_of_features)
        )
        offset_y_weight = tf.get_variable(
            name="offset_y_weight",
            initializer=tf.random_normal_initializer(stddev=tf.sqrt(1.0 / number_of_input_features)),
            dtype=tf.float32,
            shape=(number_of_experts, number_of_input_features, number_of_features)
        )
        offset_x_bias = tf.get_variable(
            name="offset_x_bias",
            initializer=offset_initializer(),
            dtype=tf.float32,
            shape=(number_of_experts, 1, number_of_features),
        )
        offset_y_bias = tf.get_variable(
            name="offset_y_bias",
            initializer=offset_initializer(),
            dtype=tf.float32,
            shape=(number_of_experts, 1,  number_of_features),
        )
#         def scale_initializer():
#             return tf.sqrt(weight_scale / number_of_input_features)
#         score_scale = tf.get_variable(
#             name="score_scale",
#             initializer=scale_initializer(),
#             dtype=tf.float32
#         )
#         offset_x_scale = tf.get_variable(
#             name="offset_x_scale",
#             initializer=scale_initializer(),
#             dtype=tf.float32
#         )
#         offset_y_scale = tf.get_variable(
#             name="offset_y_scale",
#             initializer=scale_initializer(),
#             dtype=tf.float32
#         )
        
        dynamic_image_shape = tf.shape(images)
        images_3d = tf.reshape(
            images,
            (
                number_of_experts,
                dynamic_image_shape[1] * dynamic_image_shape[2] * dynamic_image_shape[3],
                number_of_input_features
            )
        )
        def images_xw_plus_b(w, b):
            tf.matmul(images_3d, w) + b
        
        def to_4d(images):
            tf.assert_equal(tf.shape(images), (number_of_experts, dynamic_image_shape[1] * dynamic_image_shape[2] * dynamic_image_shape[3], number_of_features))
            return tf.reshape(
                images,
                (
                    number_of_experts * dynamic_image_shape[1],
                    dynamic_image_shape[2],
                    dynamic_image_shape[3],
                    number_of_features
                )
            )
        return tf.reshape(
            grid_sample(
                to_4d(tf.matmul(images_3d, score_weight)),
                to_4d(tf.matmul(images_3d, offset_x_weight) + offset_x_bias),
                to_4d(tf.matmul(images_3d, offset_y_weight) + offset_y_bias)
            ),
            (
                number_of_experts,
                dynamic_image_shape[1],
                dynamic_image_shape[2],
                dynamic_image_shape[3],
                number_of_features
            )
        )        

In [22]:
def model_fn(features, labels, mode, params, config):
    random.seed(number_of_features_per_layer * number_of_layers_per_block * number_of_blocks)
    padded_input = tf.expand_dims(
        tf.expand_dims(
            tf.pad(tf.cast(features, tf.float32) / 255.0 - 0.5, paddings),
            axis=0
        ),
        axis=4
    )
    
    features = functools.reduce(
        lambda images, i: offnet(
            functools.reduce(
                lambda features, i: tf.concat(
                    (
                        images,
                        tf.layers.batch_normalization(
                            offnet(images, number_of_features_per_layer, number_of_experts),
                            training=mode is tf.estimator.ModeKeys.TRAIN
                        )
                    ),
                    axis=4
                ),
                range(number_of_layers_per_block),
                images,
            ),
            number_of_features_of_bottleneck,
            number_of_experts
        ),
        range(number_of_blocks),
        padded_input
    )
    
    score_layer = offnet(features, number_of_classes, number_of_experts)

    scores = tf.reduce_mean(
        score_layer[:, :, 16, 16, :],
        axis=0
    )

    probabilities = tf.nn.softmax(logits=scores)
    predicted_classes = tf.argmax(scores, 1)
    predictions = {
        'probabilities' : probabilities,
        'scores': scores,
        'class': predicted_classes,
    }

    optimizer = tf.train.AdamOptimizer(epsilon=0.1)
    if labels is None:
        return tf.estimator.EstimatorSpec(
            mode=mode,
            predictions=predictions,
        )
    else:
        loss = tf.losses.softmax_cross_entropy(logits=scores, onehot_labels=tf.one_hot(labels, number_of_classes))
        eval_metric_ops = {
            'accuracy': tf.metrics.accuracy(labels=labels, predictions=predicted_classes)
        }
        update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
        with tf.control_dependencies(update_ops):
            return tf.estimator.EstimatorSpec(
                mode=mode,
                predictions=predictions,
                loss=loss,
                train_op=optimizer.minimize(loss, global_step=tf.train.get_global_step()),
                eval_metric_ops=eval_metric_ops,
            )

In [23]:
run_config = tf.estimator.RunConfig(
    model_dir=f"models/bottleneckoffnet{number_of_experts}x{number_of_features_per_layer}x{number_of_layers_per_block}x{number_of_blocks}",
    session_config=tf.ConfigProto(
        gpu_options=tf.GPUOptions(
            allow_growth=True,
        ),
        graph_options=tf.GraphOptions(
            optimizer_options=tf.OptimizerOptions(
                global_jit_level=tf.OptimizerOptions.ON_2,
                do_function_inlining=True,
                do_constant_folding=True,
                do_common_subexpression_elimination=True,
            ),
        )
    )
)

In [24]:
estimator = tf.estimator.Estimator(model_fn, config=run_config)

INFO:tensorflow:Using config: {'_model_dir': 'models/offnet1x8x5x3', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': gpu_options {
  allow_growth: true
}
graph_options {
  optimizer_options {
    do_common_subexpression_elimination: true
    do_constant_folding: true
    do_function_inlining: true
    global_jit_level: ON_2
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f81cf278f28>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [25]:
batch_size = 32

In [26]:
def training_dataset():
    return tf.data.Dataset.from_tensor_slices(training_data).shuffle(1000).batch(batch_size)

In [27]:
def testing_dataset():
    return tf.data.Dataset.from_tensor_slices(testing_data).batch(batch_size)

In [28]:
# hook = tf_debug.TensorBoardDebugHook("localhost:6064")
# estimator.train(training_dataset,hooks=[hook])

In [ ]:
tf.estimator.train_and_evaluate(
    estimator,
    train_spec=tf.estimator.TrainSpec(training_dataset),
    eval_spec=tf.estimator.EvalSpec(testing_dataset, throttle_secs=1800)
)

INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after 1800 secs (eval_spec.throttle_secs) or training is finished.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into models/offnet1x8x5x3/model.ckpt.
INFO:tensorflow:loss = 2.9007828, step = 0
INFO:tensorflow:global_step/sec: 3.20163
INFO:tensorflow:loss = 1.979384, step = 100 (31.234 sec)
INFO:tensorflow:global_step/sec: 3.50959
INFO:tensorflow:loss = 1.5797501, step = 200 (28.493 sec)
INFO:tensorflow:global_step/sec: 3.5087
INFO:tensorflow:loss = 1.2782334, step = 300 (28.501 sec)
INFO:tensorflow:global_step/sec: 3.52153
INFO:tensorflow:loss = 0.9823962, step = 400 (28.397 sec)
INFO:tensorflow:global_step/sec: 

In [ ]:
# estimator.evaluate(input_fn=lambda:tf.data.Dataset.from_tensor_slices(testing_data).batch(3).take(1))

In [ ]:
# tuple(estimator.predict(input_fn=lambda:tf.data.Dataset.from_tensor_slices(testing_data).batch(3).take(1)))

In [ ]:
# estimator.train(input_fn=training_dataset, steps=1)